
<h1><b><p style="background-image: url(https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS_i55GJPQukV5qBic4yhFxHSJaE3pBgicerQ&s);font-family:tahoma;font-size:100%;color:Navy;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Air Passengers <span style="color: red; font-size: 16px;">Time Series</span>
⏰⌛️<span style="color: red;">| </span>LSTM<span style="color: red;">| </span>Prophet<span style="color: red;">| </span>SRIMA</p></b></h1>

<img src="https://th.bing.com/th/id/OIG3.Id96Tswq65PfC93qNKfk?pid=ImgGn" style="width:60%">

<h3 style="font-family: 'poppins'; font-weight:bold ; color: Navy;">Hi 👋 everyone! Welcome to my notebook 📓. I'm <span style="color: red;">Danish Mubashar</span> and passionate about data science, and I'm excited to share my findings with you. 🤓 In my recent notebook, I successfully implemented <span style="color: red;">Time Series Air Passengers</span>using <span style="color: red;">LSTM, Prophet and SARIMA</span> . If you find this notebook helpful, please <span style="color: red;">|| UPVOTE ||</span> ❤️</h3>

<span style="color:red; font-size: 200%; font-weight: bold;">Table of Contents</span>

| **Section**     | 
|-----------------|
| [Libraries](#1) | 
| [EDA](#2)       |
| [LSTM](#3)      | 
| [Prophet](#4)   | 
| [SARIMA](#5)    | 


<a id="1"></a>
<h1><b><p style="background-image: url(https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS_i55GJPQukV5qBic4yhFxHSJaE3pBgicerQ&s);font-family:tahoma;font-size:120%;color:Navy;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Libraries</p></b></h1>

In [ ]:
# import libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings


<a id="2"></a>
<h1><b><p style="background-image: url(https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS_i55GJPQukV5qBic4yhFxHSJaE3pBgicerQ&s);font-family:tahoma;font-size:120%;color:Navy;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">EDA</p></b></h1>

<span style="color: navy; font-size: 25px; font-weight: bold;">Step 1 || Read Data</span>


In [ ]:
# load the data
df = pd.read_csv('/kaggle/input/air-passengers/AirPassengers.csv')
df.head()

<span style="color: navy; font-size: 25px; font-weight: bold;">Step 2 || Rename Data</span>


In [ ]:
# change the name of both columns
df.rename(columns = {'Month':'Date'}, inplace = True)
df.rename(columns = {'#Passengers':'Number of Passengers'}, inplace = True)

<span style="color: navy; font-size: 25px; font-weight: bold;">Step 3 || Information</span>


In [ ]:
df.info()

<span style="color: navy; font-size: 25px; font-weight: bold;">Step 4 || Statistical Summery</span>


In [ ]:
df.describe()

<span style="color: navy; font-size: 25px; font-weight: bold;">Step 5 || Seasonal Decompose</span>


In [ ]:
# Perform decomposition
result = seasonal_decompose(df['Number of Passengers'], model='additive', period=12)  # Assuming monthly data with period=12

# Create subplots
fig = make_subplots(rows=4, cols=1, subplot_titles=('Original', 'Trend', 'Seasonal', 'Residual'))

# Add traces for each component
fig.add_trace(go.Scatter(x=df['Date'], y=result.observed, mode='lines', name='Original', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=df['Date'], y=result.trend, mode='lines', name='Trend', line=dict(color='green')), row=2, col=1)
fig.add_trace(go.Scatter(x=df['Date'], y=result.seasonal, mode='lines', name='Seasonal', line=dict(color='red')), row=3, col=1)
fig.add_trace(go.Scatter(x=df['Date'], y=result.resid, mode='lines', name='Residual', line=dict(color='orange')), row=4, col=1)

# Update layout
fig.update_layout(title='Time Series Decomposition', height=800, showlegend=False, plot_bgcolor='lightgrey')

# Add animation
frames = []
for i in range(len(df)):
    frame = go.Frame(data=[go.Scatter(x=df['Date'][:i+1], y=result.observed[:i+1], mode='lines', name='Original', line=dict(color='blue')),
                            go.Scatter(x=df['Date'][:i+1], y=result.trend[:i+1], mode='lines', name='Trend', line=dict(color='green')),
                            go.Scatter(x=df['Date'][:i+1], y=result.seasonal[:i+1], mode='lines', name='Seasonal', line=dict(color='red')),
                            go.Scatter(x=df['Date'][:i+1], y=result.resid[:i+1], mode='lines', name='Residual', line=dict(color='orange'))],
                      traces=[1, 2, 3, 4])
    frames.append(frame)

# Update frames
fig.frames = frames

# Update play/pause button
fig.update_layout(updatemenus=[dict(type="buttons", buttons=[dict(label="Play", method="animate", args=[None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True, "mode": "immediate"}]),
                                                              dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}])])])

# Show plot
fig.show()


<a id="3"></a>
<h1><b><p style="background-image: url(https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS_i55GJPQukV5qBic4yhFxHSJaE3pBgicerQ&s);font-family:tahoma;font-size:120%;color:Navy;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">LSTM</p></b></h1>

<h3 style="font-family: 'poppins'; font-weight:bold ; color: Navy;">LSTM stands for Long Short-Term Memory, and it is a type of recurrent neural network (RNN) architecture that is designed to handle long-term dependencies in sequential data. 🧠💭 It was introduced by Hochreiter and Schmidhuber in 1997.

By using the input gate, forget gate, and output gate, LSTM networks can selectively store and retrieve information over long sequences. 🚪🔒 This allows them to capture long-term dependencies and mitigate the vanishing gradient problem. As a result, LSTMs have been widely used in various applications such as natural language processing, speech recognition, and time series analysis. 📚🗣️⌛</h3>

In [ ]:

# Assuming df is your DataFrame with 'Date' and 'Number of Passengers' columns

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Sort DataFrame by date
df.sort_values(by='Date', inplace=True)

# Set 'Date' column as index
df.set_index('Date', inplace=True)

# Convert DataFrame to numpy array
data = df.values

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Define window size (number of time steps to look back)
window_size = 12

# Function to create dataset with input features and target variable
def create_dataset(data, window_size):
    X, Y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:(i + window_size)])
        Y.append(data[i + window_size])
    return np.array(X), np.array(Y)

# Create dataset
X, Y = create_dataset(scaled_data, window_size)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

# Reshape data for LSTM (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [ ]:

# Define LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate model
loss = model.evaluate(X_test, y_test)
print('Test Loss:', loss)

In [ ]:
# Make predictions
predictions = model.predict(X_test)

# Inverse transform predictions and actual values
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

# Plot actual vs. predicted values
plt.figure(figsize=(12, 6))
plt.plot(df.index[-len(y_test):], y_test, label='Actual')
plt.plot(df.index[-len(predictions):], predictions, label='Predicted')
plt.xlabel('Date')
plt.ylabel('Number of Passengers')
plt.title('Actual vs. Predicted Number of Passengers')
plt.legend()
plt.grid(True)
plt.show()


<a id="4"></a>
<h1><b><p style="background-image: url(https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS_i55GJPQukV5qBic4yhFxHSJaE3pBgicerQ&s);font-family:tahoma;font-size:120%;color:Navy;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Prophet</p></b></h1>

<h3 style="font-family: 'poppins'; font-weight:bold ; color: Navy;">The Prophet model is an open-source algorithm for time-series analysis introduced by Facebook, designed for forecasting daily data with weekly and yearly seasonality, holidays, and custom seasonalities. ⏰📈📅

Prophet is widely used for time series forecasting tasks due to its ease of use, flexibility, and ability to handle various patterns and phenomena in the data. 🧩🔍💪 It has been applied in domains such as retail sales forecasting, demand planning, financial forecasting, and capacity planning, among others. 🛍️📊💼</h3>

In [ ]:
from prophet import Prophet
import plotly.graph_objs as go

# Prepare the DataFrame for Prophet
prophet_df = df.reset_index().rename(columns={'Date': 'ds', 'Number of Passengers': 'y'})

# Initialize Prophet model
model = Prophet()

# Fit the model
model.fit(prophet_df)

# Make future predictions
future = model.make_future_dataframe(periods=365)  # Predicting for 1 year into the future
forecast = model.predict(future)

# Calculate Mean Squared Error
mse = ((forecast['yhat'] - prophet_df['y']) ** 2).mean()
print('Mean Squared Error:', mse)

# Plot the forecast
fig = go.Figure()

# Plot observed data
fig.add_trace(go.Scatter(x=prophet_df['ds'], y=prophet_df['y'], mode='lines', name='Observed'))

# Plot forecast
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast'))

# Add layout
fig.update_layout(title='Forecast using Prophet Model',
                  xaxis_title='Date',
                  yaxis_title='Number of Passengers')

# Show plot
fig.show()


In [ ]:
# Plot the forecast
fig = model.plot(forecast)
plt.xlabel('Date')
plt.ylabel('Number of Passengers')
plt.title('Forecast using Prophet Model')
plt.show()


<a id="5"></a>
<h1><b><p style="background-image: url(https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS_i55GJPQukV5qBic4yhFxHSJaE3pBgicerQ&s);font-family:tahoma;font-size:120%;color:Navy;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">SARIMA</p></b></h1>

<h3 style="font-family: 'poppins'; font-weight:bold ; color: Navy;">The SARIMA (Seasonal Autoregressive Integrated Moving Average) model is an extension of the ARIMA model designed to handle time series data with seasonal patterns. 📈🔄🌙 It incorporates seasonal components to capture recurring patterns at fixed intervals, such as daily, weekly, or yearly cycles. By introducing seasonal autoregressive, moving average, and integration terms, SARIMA can effectively model and forecast time series data with complex seasonal patterns. 📊🔄🌙

The model requires determining the order of the seasonal components (P, D, Q, s) 📝

where:

'P' represents the seasonal autoregressive order, 🔄\
'D' represents the seasonal differencing order, 📉\
'Q' represents the seasonal moving average order, 🔄\
's' represents the number of observations in each seasonal period. 🌙</h3>

<span style="color: navy; font-size: 25px; font-weight: bold;">Step 1 || Find Good Value Of P</span>


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf

# Plot ACF
fig, ax = plt.subplots(figsize=(12, 6))
plot_acf(df['Number of Passengers'], lags=20, alpha=0.05, ax=ax)
ax.set_xlabel('Lag')
ax.set_ylabel('Autocorrelation')
ax.set_title('Autocorrelation Plot (ACF)')
ax.grid(True)

# Add autocorrelation values on each bar
acf_values = acf(df['Number of Passengers'], nlags=20, fft=False)
for lag, acf_value in enumerate(acf_values):
    ax.text(lag, acf_value, f'{acf_value:.2f}', ha='center', va='bottom', fontsize=8, color='black')

plt.show()


 <span style="color: red;"> Good Value for P is (1)</span>


In [ ]:
from statsmodels.tsa.stattools import acf
x_acf = pd.DataFrame(acf(df['Number of Passengers']))
print(x_acf)


<span style="color: navy; font-size: 25px; font-weight: bold;">Step 2 || Find Good Value Of D </span>


In [ ]:

# auttocorrelation check difference values
from statsmodels.graphics.tsaplots import plot_acf
fig, ax = plt.subplots(3,2, sharex=True)
ax[0,0].plot(df['Number of Passengers'])
ax[0,0].set_title('Original Series')
plot_acf(df['Number of Passengers'], ax=ax[0,1])

# 1st Differencing
ax[1,0].plot(df['Number of Passengers'].diff()); ax[1,0].set_title('1st Order Differencing')
plot_acf(df['Number of Passengers'].diff().dropna(), ax=ax[1,1])

# 2nd Differencing
ax[2,0].plot(df['Number of Passengers'].diff().diff()); ax[2,0].set_title('2nd Order Differencing')
plot_acf(df['Number of Passengers'].diff().diff().dropna(), ax=ax[2,1])

 <span style="color: red;"> Good Value for D is (1)</span>


<span style="color: navy; font-size: 25px; font-weight: bold;">Step 3 || Find Good Value Of Q </span>


In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

# Plot PACF
plt.figure(figsize=(12, 6))
plot_pacf(df['Number of Passengers'], lags=20)
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.title('Partial Autocorrelation Plot (PACF)')
plt.grid(True)
plt.show()


 <span style="color: red;"> Good Value for Q is (3)</span>


In [ ]:
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
p, d, q = 1, 1, 3
model = sm.tsa.statespace.SARIMAX(df['Number of Passengers'], order=(p, d, q), seasonal_order=(p, d, q, 12))
model = model.fit()
# print summary
print(model.summary())


In [ ]:
# prediction
start_date = df.index[-1]  # Get the last date in the DataFrame
end_date = start_date + pd.DateOffset(days=365)  # Predict for the next 10 days

predictions = model.predict(start=start_date, end=end_date)


In [ ]:
import plotly.graph_objs as go

# Create traces
trace_observed = go.Scatter(x=df.index, y=df['Number of Passengers'], mode='lines', name='Observed')
trace_predicted = go.Scatter(x=predictions.index, y=predictions, mode='lines', name='Predicted')

# Create layout
layout = go.Layout(title='SARIMA Model',
                   xaxis=dict(title='Date'),
                   yaxis=dict(title='Number of Passengers'))

# Combine traces into a figure
fig = go.Figure(data=[trace_observed, trace_predicted], layout=layout)

# Show plot
fig.show()